In [1]:
import requests
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize

In [3]:
#get season
season_info_raw = requests.get(base_football_url + liga+lang).json()
data_season = season_info_raw['phases']

In [4]:
# Normalize the JSON data
normalized_data = json_normalize(data_season)

# Convert the normalized data to a Pandas DataFrame
df_season_raw = pd.DataFrame(normalized_data)

# View the DataFrame
df_season_raw.head()

/var/folders/zy/65zqs5x55kd6l4xmdbksz289zrrqyz/T/ipykernel_4276/331063180.py:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  normalized_data = json_normalize(data_season)


,id,key,displayName,state,isActual,showActuals,type,cesimReference.contestId,cesimReference.masterEventId,cesimReference.eventId,sport.id,sport.key,sport.name
0,4693-171-160,4693-171-160,1. Runde,Finished,False,False,Round,4693,171,160,7,football,Fussball
1,4693-171-161,4693-171-161,2. Runde,Finished,False,False,Round,4693,171,161,7,football,Fussball
2,4693-171-162,4693-171-162,3. Runde,Finished,False,False,Round,4693,171,162,7,football,Fussball
3,4693-171-163,4693-171-163,4. Runde,Finished,False,False,Round,4693,171,163,7,football,Fussball
4,4693-171-164,4693-171-164,5. Runde,Finished,False,False,Round,4693,171,164,7,football,Fussball


In [5]:
#get the season id (for now singular)
df_season_raw['season_id'] = df_season_raw['cesimReference.contestId'].astype(str) + "-" + df_season_raw['cesimReference.masterEventId'].astype(str)
season_ids = list(set(df_season_raw['season_id']))
season_id = season_ids[0]

In [6]:
#get all the games for this season
base_url_eventitems = "https://test.sport.api.swisstxt.ch/v1/eventItems?phaseIds="
season_id = season_ids[0]
start_date = "2016-01-01" #we don't care as long as it includes our chosen season above
end_date = "2030-01-01"
games_info_raw = requests.get(base_url_eventitems + season_id +"&fromDate="+start_date +"&toDate="+end_date+lang).json()
finished_rounds = list(df_season_raw[df_season_raw['state']=="Finished"]['displayName'])

In [7]:
#extract the nested info about the phase, as the normalization below doesn't get this one
container = []

for i in range(0,len(games_info_raw)):
    game = {'id': games_info_raw[i]['id'],
            'round_id': games_info_raw[i]['contestInfo']['phases'][1]['id']}
    container.append(game)

phase_info = pd.DataFrame(container)
    

In [8]:
# Normalize the JSON data
normalized_data = json_normalize(games_info_raw)

# Convert the normalized data to a Pandas DataFrame
df_games_raw = pd.DataFrame(normalized_data)


/var/folders/zy/65zqs5x55kd6l4xmdbksz289zrrqyz/T/ipykernel_4276/3859984260.py:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  normalized_data = json_normalize(games_info_raw)


In [9]:
#drop all the columns in Italian and French
cols_to_drop = [col for col in df_games_raw.columns if col.endswith('.it') or col.endswith('.fr')]
df_games_raw.drop(cols_to_drop, axis=1, inplace=True)
pd.set_option('display.max_columns', None)

df_games_raw = pd.merge(df_games_raw, phase_info, on='id')

df_games_raw.head()

,id,type,state,eventItemOrder,pushLiveData,dateTimeInfo.fullDateTime,dateTimeInfo.date,dateTimeInfo.time,displayTitle.de,sport.id,sport.key,sport.name.de,competitor1.id,competitor1.infostradaId,competitor1.name.de,competitor1.shortName.de,competitor1.country,competitor1.isNationalTeam,competitor1.results.main,competitor1.results.firstHalf,competitor1.results.secondHalf,competitor2.id,competitor2.infostradaId,competitor2.name.de,competitor2.shortName.de,competitor2.country,competitor2.isNationalTeam,competitor2.results.main,competitor2.results.firstHalf,competitor2.results.secondHalf,videoAssetId.de,videoUrn.de,contestInfo.contestSeason.id,contestInfo.contestSeason.key,contestInfo.contestSeason.name.de,contestInfo.contestSeason.season,contestInfo.phases,scores.firstHalf.competitor1,scores.firstHalf.competitor2,scores.firstHalf.formatted.de,scores.secondHalf.competitor1,scores.secondHalf.competitor2,scores.secondHalf.formatted.de,scores.main.competitor1,scores.main.competitor2,scores.main.formatted.de,city.de,stadium.de,spectators.de,referees.de,stateDetail,round_id
0,1752045,Game,Finished,0,True,2022-07-16T18:00:00,Sa 16.07.,18:00,BSC Young Boys - FC Zürich,7,football,Fussball,2758,4281,BSC Young Boys,YB,SUI,False,4.0,0.0,4.0,2688,4993,FC Zürich,Zürich,SUI,False,0.0,0.0,0.0,56e5abbb-e71f-42d4-8e13-e704c99bd1de,urn:srf:video:56e5abbb-e71f-42d4-8e13-e704c99b...,4693,super-league,Super League,2023,"[{'id': '4693-171', 'key': '4693-171', 'name':...",0.0,0.0,0 : 0,4.0,0.0,4 : 0,4.0,0.0,4 : 0,Bern,Wankdorf,31120,Sandro Schärer,NaN,4693-171-160
1,1752046,Game,Finished,0,True,2022-07-16T20:30:00,Sa 16.07.,20:30,FC Winterthur - FC Basel 1893,7,football,Fussball,2766,103269,FC Winterthur,Winterthur,SUI,False,1.0,1.0,0.0,2689,4015,FC Basel 1893,Basel,SUI,False,1.0,0.0,1.0,4f73ca6a-0593-4252-a9d4-478ad304f919,urn:srf:video:4f73ca6a-0593-4252-a9d4-478ad304...,4693,super-league,Super League,2023,"[{'id': '4693-171', 'key': '4693-171', 'name':...",1.0,0.0,1 : 0,0.0,1.0,0 : 1,1.0,1.0,1 : 1,Winterthur,Schützenwiese,8400,Alain Bieri,NaN,4693-171-160
2,1752047,Game,Finished,0,True,2022-07-17T16:30:00,So 17.07.,16:30,FC Lugano - FC Sion,7,football,Fussball,14099,113638,FC Lugano,Lugano,SUI,False,2.0,0.0,2.0,2690,4116,FC Sion,Sion,SUI,False,3.0,1.0,2.0,fc976816-873b-4661-8325-e89438be23a8,urn:srf:video:fc976816-873b-4661-8325-e89438be...,4693,super-league,Super League,2023,"[{'id': '4693-171', 'key': '4693-171', 'name':...",0.0,1.0,0 : 1,2.0,2.0,2 : 2,2.0,3.0,2 : 3,Lugano,Cornaredo,2737,Fedayi San,NaN,4693-171-160
3,1752048,Game,Finished,0,True,2022-07-17T16:30:00,So 17.07.,16:30,Servette FC - FC St.Gallen 1879,7,football,Fussball,2685,4113,Servette FC,Servette,SUI,False,1.0,1.0,0.0,2684,100090,FC St.Gallen 1879,St.Gallen,SUI,False,0.0,0.0,0.0,8863af66-2ff3-4b2c-81fd-5a9c46ed0783,urn:srf:video:8863af66-2ff3-4b2c-81fd-5a9c46ed...,4693,super-league,Super League,2023,"[{'id': '4693-171', 'key': '4693-171', 'name':...",1.0,0.0,1 : 0,0.0,0.0,0 : 0,1.0,0.0,1 : 0,Genève,Stade de Genève,4872,Lukas Fähndrich,NaN,4693-171-160
4,1752049,Game,Finished,0,True,2022-08-10T19:00:00,Mi 10.08.,19:00,FC Luzern - Grasshopper Club Zürich,7,football,Fussball,2679,4079,FC Luzern,Luzern,SUI,False,1.0,1.0,0.0,2681,4058,Grasshopper Club Zürich,GC,SUI,False,1.0,1.0,0.0,02c7d8ee-ac9e-424f-8d39-2929158fa5e2,urn:srf:video:02c7d8ee-ac9e-424f-8d39-2929158f...,4693,super-league,Super League,2023,"[{'id': '4693-171', 'key': '4693-171', 'name':...",1.0,1.0,1 : 1,0.0,0.0,0 : 0,1.0,1.0,1 : 1,Luzern,swissporarena,12919,Alessandro Dudic,NaN,4693-171-160


In [10]:
#get a list of all the teams of this season
all_teams = list(set(df_games_raw["competitor1.name.de"]))

In [11]:
# Create a list of dictionaries for each finished game with the round ID, competitor names, and scores
games = []
for i in range(len(df_games_raw)):
    if df_games_raw.loc[i, 'state'] == 'Finished':
        game = {}
        game['round_id'] = df_games_raw.loc[i, 'round_id']
        game['team1'] = df_games_raw.loc[i, 'competitor1.name.de']
        game['team2'] = df_games_raw.loc[i, 'competitor2.name.de']
        game['team1_score'] = df_games_raw.loc[i, 'competitor1.results.main']
        game['team2_score'] = df_games_raw.loc[i, 'competitor2.results.main']
        games.append(game)

# Create a list of dictionaries for each team with the team name, round ID, points, goals scored, and goals scored against
teams = []
for team in all_teams:
    for i, round_id in enumerate(df_games_raw['round_id'].unique(), start=1):
        team_data = {'team': team, 'round_id': round_id, 'points': 0, 'goals_scored': 0, 'goals_conceded': 0, 'round_num': i}
        teams.append(team_data)

# Calculate the points and other stats for each team in each round based on their scores in the games
for game in games:
    team1_goals = game['team1_score']
    team2_goals = game['team2_score']
    team1_points = 1
    team2_points = 1
    if team1_goals > team2_goals:
        team1_points = 3
        team2_points = 0
    elif team1_goals < team2_goals:
        team1_points = 0
        team2_points = 3
    for team in teams:
        if team['team'] == game['team1'] and team['round_id'] == game['round_id']:
            team['points'] += team1_points
            team['goals_scored'] += team1_goals
            team['goals_conceded'] += team2_goals
        elif team['team'] == game['team2'] and team['round_id'] == game['round_id']:
            team['points'] += team2_points
            team['goals_scored'] += team2_goals
            team['goals_conceded'] += team1_goals

# Create a new DataFrame from the list of team dictionaries
df_stats = pd.DataFrame(teams)


In [12]:
# Add up points for each team one round after another
df_stats.sort_values(['team', 'round_num'], inplace=True)
df_stats['total_points'] = df_stats.groupby('team')['points'].cumsum()

# Calculate average points scored for last 5 rounds
df_stats['avg_points_last_5'] = df_stats.groupby('team')['points'].rolling(5, min_periods=1).mean().reset_index(0, drop=True)

# Calculate average points scored for last 10 rounds
df_stats['avg_points_last_10'] = df_stats.groupby('team')['points'].rolling(10, min_periods=1).mean().reset_index(0, drop=True)

# Calculate average goals scored for last 5 rounds
df_stats['avg_goals_scored_last_5'] = df_stats.groupby('team')['goals_scored'].rolling(5, min_periods=1).mean().reset_index(0, drop=True)

# Calculate average goals scored for last 10 rounds
df_stats['avg_goals_scored_last_10'] = df_stats.groupby('team')['goals_scored'].rolling(10, min_periods=1).mean().reset_index(0, drop=True)

# Calculate average goals conceded for last 5 rounds
df_stats['avg_goals_conceded_last_5'] = df_stats.groupby('team')['goals_conceded'].rolling(5, min_periods=1).mean().reset_index(0, drop=True)

# Calculate average goals conceded for last 10 rounds
df_stats['avg_goals_conceded_last_10'] = df_stats.groupby('team')['goals_conceded'].rolling(10, min_periods=1).mean().reset_index(0, drop=True)

# Sort by round_num
df_stats.sort_values(['team','round_num'], inplace=True)
df_stats = df_stats.reset_index()


In [13]:
df_season_small = df_season_raw[["id", "displayName"]]
df_season_small.columns = ["round_id", "displayName"]
df_stats = pd.merge(df_stats, df_season_small)


In [14]:
# Number of rounds left
df_stats['num_rounds_left'] = max(df_stats['round_num'])-df_stats['round_num']
df_stats.head()

,index,team,round_id,points,goals_scored,goals_conceded,round_num,total_points,avg_points_last_5,avg_points_last_10,avg_goals_scored_last_5,avg_goals_scored_last_10,avg_goals_conceded_last_5,avg_goals_conceded_last_10,displayName,num_rounds_left
0,216,BSC Young Boys,4693-171-160,3,4.0,0.0,1,3,3.0,3.0,4.0,4.0,0.0,0.0,1. Runde,35
1,108,FC Basel 1893,4693-171-160,1,1.0,1.0,1,1,1.0,1.0,1.0,1.0,1.0,1.0,1. Runde,35
2,288,FC Lugano,4693-171-160,0,2.0,3.0,1,0,0.0,0.0,2.0,2.0,3.0,3.0,1. Runde,35
3,180,FC Luzern,4693-171-160,1,1.0,1.0,1,1,1.0,1.0,1.0,1.0,1.0,1.0,1. Runde,35
4,144,FC Sion,4693-171-160,3,3.0,2.0,1,3,3.0,3.0,3.0,3.0,2.0,2.0,1. Runde,35


In [15]:
#get the projected averages
proj_points_avg_last_5 = df_stats['total_points'] + (df_stats['num_rounds_left'] * df_stats['avg_points_last_5'])
df_stats['proj_points_avg_last_5'] = proj_points_avg_last_5
proj_points_avg_last_10 = df_stats['total_points'] + (df_stats['num_rounds_left'] * df_stats['avg_points_last_10'])
df_stats['proj_points_avg_last_10'] = proj_points_avg_last_10
df_stats.head()


,index,team,round_id,points,goals_scored,goals_conceded,round_num,total_points,avg_points_last_5,avg_points_last_10,avg_goals_scored_last_5,avg_goals_scored_last_10,avg_goals_conceded_last_5,avg_goals_conceded_last_10,displayName,num_rounds_left,proj_points_avg_last_5,proj_points_avg_last_10
0,216,BSC Young Boys,4693-171-160,3,4.0,0.0,1,3,3.0,3.0,4.0,4.0,0.0,0.0,1. Runde,35,108.0,108.0
1,108,FC Basel 1893,4693-171-160,1,1.0,1.0,1,1,1.0,1.0,1.0,1.0,1.0,1.0,1. Runde,35,36.0,36.0
2,288,FC Lugano,4693-171-160,0,2.0,3.0,1,0,0.0,0.0,2.0,2.0,3.0,3.0,1. Runde,35,0.0,0.0
3,180,FC Luzern,4693-171-160,1,1.0,1.0,1,1,1.0,1.0,1.0,1.0,1.0,1.0,1. Runde,35,36.0,36.0
4,144,FC Sion,4693-171-160,3,3.0,2.0,1,3,3.0,3.0,3.0,3.0,2.0,2.0,1. Runde,35,108.0,108.0


In [16]:
# Calculate goal difference for each game
df_stats['goal_diff_game'] = df_stats['goals_scored'] - df_stats['goals_conceded']

# Group by round and sum goal differences
df_stats['goal_diff'] = df_stats.groupby('team')['goal_diff_game'].cumsum()

#Rank now for each round
df_stats['rank'] = df_stats.sort_values(by=['total_points', 'goal_diff'],
               ascending=[False, False]).groupby('round_num').cumcount() + 1
df_stats['rank_proj5'] = df_stats.sort_values(by=['proj_points_avg_last_5', 'goal_diff'],
               ascending=[False, False]).groupby('round_num').cumcount() + 1
df_stats['rank_proj10'] = df_stats.sort_values(by=['proj_points_avg_last_10', 'goal_diff'],
               ascending=[False, False]).groupby('round_num').cumcount() + 1

df_stats.head()

,index,team,round_id,points,goals_scored,goals_conceded,round_num,total_points,avg_points_last_5,avg_points_last_10,avg_goals_scored_last_5,avg_goals_scored_last_10,avg_goals_conceded_last_5,avg_goals_conceded_last_10,displayName,num_rounds_left,proj_points_avg_last_5,proj_points_avg_last_10,goal_diff_game,goal_diff,rank,rank_proj5,rank_proj10
0,216,BSC Young Boys,4693-171-160,3,4.0,0.0,1,3,3.0,3.0,4.0,4.0,0.0,0.0,1. Runde,35,108.0,108.0,4.0,4.0,1,1,1
1,108,FC Basel 1893,4693-171-160,1,1.0,1.0,1,1,1.0,1.0,1.0,1.0,1.0,1.0,1. Runde,35,36.0,36.0,0.0,0.0,4,4,4
2,288,FC Lugano,4693-171-160,0,2.0,3.0,1,0,0.0,0.0,2.0,2.0,3.0,3.0,1. Runde,35,0.0,0.0,-1.0,-1.0,8,8,8
3,180,FC Luzern,4693-171-160,1,1.0,1.0,1,1,1.0,1.0,1.0,1.0,1.0,1.0,1. Runde,35,36.0,36.0,0.0,0.0,5,5,5
4,144,FC Sion,4693-171-160,3,3.0,2.0,1,3,3.0,3.0,3.0,3.0,2.0,2.0,1. Runde,35,108.0,108.0,1.0,1.0,2,2,2


In [17]:
df_stats[['rank_proj10','team']][df_stats['displayName']=="29. Runde"].sort_values('rank_proj10')

,rank_proj10,team
280,1,BSC Young Boys
283,2,FC Luzern
289,3,Servette FC
282,4,FC Lugano
288,5,Grasshopper Club Zürich
281,6,FC Basel 1893
287,7,FC Zürich
285,8,FC St.Gallen 1879
284,9,FC Sion
286,10,FC Winterthur


In [18]:
df_stats[df_stats['displayName'].isin(finished_rounds)].to_csv('super_league.csv', index=False)

0       True
1       True
2       True
3       True
4       True
       ...  
175    False
176    False
177    False
178    False
179    False
Name: state, Length: 180, dtype: bool